In [1]:
# Mobile BERT: BERT를 압축하고 속도를 개선한 모형

In [2]:
from transformers import MobileBertTokenizer, MobileBertModel
import torch

# 모델 및 토크나이저 불러오기
tokenizer_mbert = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model_mbert = MobileBertModel.from_pretrained('google/mobilebert-uncased')
model_mbert

C:\Python\Python39\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tjoeun\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


MobileBertModel(
  (embeddings): MobileBertEmbeddings(
    (word_embeddings): Embedding(30522, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (embedding_transformation): Linear(in_features=384, out_features=512, bias=True)
    (LayerNorm): NoNorm()
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (encoder): MobileBertEncoder(
    (layer): ModuleList(
      (0-23): 24 x MobileBertLayer(
        (attention): MobileBertAttention(
          (self): MobileBertSelfAttention(
            (query): Linear(in_features=128, out_features=128, bias=True)
            (key): Linear(in_features=128, out_features=128, bias=True)
            (value): Linear(in_features=512, out_features=128, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): MobileBertSelfOutput(
            (dense): Linear(in_features=128, out_features=128, bias=True)
            (LayerNorm): NoNorm()
          )
      

In [3]:
from transformers import BertTokenizer, BertModel

# 모델 및 토크나이저 불러오기
tokenizer_bert = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = BertModel.from_pretrained('bert-base-uncased')
model_bert

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [4]:
import torch
text = "Mobile bert is more practical than bert."

# Mobile BERT 토크나이징
inputs = tokenizer_mbert.tokenize(text)
print(inputs)

['mobile', 'bert', 'is', 'more', 'practical', 'than', 'bert', '.']


In [6]:
# BERT 토크나이징
inputs = tokenizer_bert.tokenize(text)
print(inputs)
# 두 모델의 실행결과가 같음

['mobile', 'bert', 'is', 'more', 'practical', 'than', 'bert', '.']


In [7]:
import torch
text = 'Mobile bert is more practical than bert.'
inputs = tokenizer_mbert.encode(text)
# squeeze() 사이즈가 1인 차원 제거, unsqeeeze() 첫번쨰 위치에 1인 차우너 추가
outputs = model_mbert(torch.tensor(inputs).unsqueeze(0))
print(outputs.last_hidden_state.shape)

inputs = tokenizer_bert.encode(text)
outputs = model_bert(torch.tensor(inputs).unsqueeze(0))
print(outputs.last_hidden_state.shape)

# torch.Size([1, 10, 512])  BERT 모형
# torch.Size([1, 10, 768])  Mobile BERT 모형

torch.Size([1, 10, 512])
torch.Size([1, 10, 768])


In [8]:
# Mobile BERT 추론
from transformers import MobileBertTokenizer, MobileBertForMaskedLM
import torch

# 토크나이저 및 모델 불러오기
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertForMaskedLM.from_pretrained('google/mobilebert-uncased')

# 마스크한 문장 및 정답 문장을 각기 토크나이징
inputs = tokenizer("The capital of Korea is [MASK].", return_tensors='pt')
labels = tokenizer("The capital of Korea is Seoul.", return_tensors='pt')['input_ids']

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

print(' '.join([tokenizer.decode(i.item()).replace(" ", "") for i in logits.argmax(-1)[0]][1:-1]))

the capital of korea is seoul .


In [11]:
# BERT 추론
from transformers import BertTokenizer, BertForMaskedLM
import torch

# 토크나이저 및 모델 불러오기
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# 마스크한 문장 및 정답 문장을 각기 토크나이징
inputs = tokenizer("The capital of Japan is [MASK].", return_tensors='pt')
labels = tokenizer("The capital of Japan is Tokyo.", return_tensors='pt')['input_ids']

outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

print(' '.join([tokenizer.decode(i.item()).replace(" ", "") for i in logits.argmax(-1)[0]][1:-1]))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


the capital of japan is tokyo .


In [12]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
model = AutoModelWithLMHead.from_pretrained('distilgpt2')
model

C:\Python\Python39\lib\site-packages\transformers\models\auto\modeling_auto.py:1423: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [15]:
# 문장완성
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')

# greedy_output = model.generate(input_ids, max_length=12)
greedy_output = model.generate(input_ids, max_length=15)

print("Output:\n" + 100 * '-')

print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good thing to have a good friend


In [16]:
from transformers import AutoTokenizer, AutoModelWithLMHead

# 토크나이저 및 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained('microsoft/DialoGPT-small')
model = AutoModelWithLMHead.from_pretrained('microsoft/DialoGPT-small')

C:\Python\Python39\lib\site-packages\transformers\models\auto\modeling_auto.py:1423: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [18]:
input_ids = tokenizer.encode("I like gpt because it's", return_tensors='pt')
greedy_output = model.generate(input_ids, max_length=30)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output:
----------------------------------------------------------------------------------------------------
I like gpt because it's a good way to get a feel for the game.
